In [4]:
import sympy as sp
from random import randint
from sympy import isprime, mod_inverse

In [5]:
def points_on_elliptic_curve(p, a, b):
    # Initialize the set of points on the curve
    points = set()
    # Loop through all possible x values to find possible points on the curve
    for x in range(p):
        # Calculate the right side of the elliptic curve equation
        rhs = (x**3 + a*x + b) % p
        # Check for y values that satisfy the equation y^2 = x^3 + ax + b
        for y in range(p):
            if pow(y, 2, p) == rhs:
                points.add((x, y))
    
    # Include the point at infinity
    points.add('infinity')
    
    return points

def hasse_theorem(p, number_of_points):
    # The Hasse theorem provides a bound for the number of points on the curve
    lower_bound = p + 1 - 2*sp.sqrt(p)
    upper_bound = p + 1 + 2*sp.sqrt(p)
    return lower_bound <= number_of_points <= upper_bound

In [6]:
def format_output(points, number_of_points, is_valid, p):
    # Initialize formatted output string
    formatted_output = "Points:\n"
    # Sort the points excluding 'infinity' for display
    sorted_points = sorted([point for point in points if point != 'infinity'])
    # Add each point to the formatted output
    for point in sorted_points:
        formatted_output += f"- {point}\n"
    # Add the point at infinity with its symbolic representation 'I'
    formatted_output += "- I\n\n"
    # Add the number of points
    formatted_output += f"#E(F_p) = {number_of_points}\n"
    # Calculate Hasse interval
    lower_bound = p + 1 - 2*sp.sqrt(p)
    upper_bound = p + 1 + 2*sp.sqrt(p)
    # Add the Hasse interval to the formatted output
    formatted_output += f"Hasse Interval = [{lower_bound.evalf()}, {upper_bound.evalf()}]\n"
    # Add validity of the Hasse theorem to the formatted output
    formatted_output += f"Hasse = {is_valid}\n"

    return formatted_output

In [7]:
# Example usage of the functions
p = 167  # a prime number
a = -3   # coefficient a in the curve equation
b = 64  # coefficient b in the curve equation

# Calculate points on the elliptic curve
points = points_on_elliptic_curve(p, a, b)

# Calculate the number of points
number_of_points = len(points)

# Verify the number of points using the Hasse theorem
is_valid = hasse_theorem(p, number_of_points)

In [8]:
# Use the previously computed points, number of points, and validity to format the output
output = format_output(points, number_of_points, is_valid, p)
print(output)

Points:
- (0, 8)
- (0, 159)
- (1, 79)
- (1, 88)
- (2, 20)
- (2, 147)
- (4, 28)
- (4, 139)
- (5, 72)
- (5, 95)
- (9, 76)
- (9, 91)
- (10, 52)
- (10, 115)
- (14, 42)
- (14, 125)
- (15, 80)
- (15, 87)
- (19, 55)
- (19, 112)
- (21, 54)
- (21, 113)
- (24, 17)
- (24, 150)
- (26, 19)
- (26, 148)
- (27, 36)
- (27, 131)
- (30, 33)
- (30, 134)
- (31, 6)
- (31, 161)
- (32, 2)
- (32, 165)
- (34, 45)
- (34, 122)
- (36, 55)
- (36, 112)
- (40, 22)
- (40, 145)
- (41, 15)
- (41, 152)
- (53, 73)
- (53, 94)
- (56, 50)
- (56, 117)
- (57, 65)
- (57, 102)
- (61, 57)
- (61, 110)
- (62, 8)
- (62, 159)
- (66, 17)
- (66, 150)
- (69, 82)
- (69, 85)
- (70, 62)
- (70, 105)
- (71, 81)
- (71, 86)
- (72, 39)
- (72, 128)
- (77, 17)
- (77, 150)
- (79, 7)
- (79, 160)
- (83, 63)
- (83, 104)
- (84, 0)
- (85, 83)
- (85, 84)
- (86, 60)
- (86, 107)
- (90, 29)
- (90, 138)
- (91, 5)
- (91, 162)
- (92, 16)
- (92, 151)
- (94, 83)
- (94, 84)
- (98, 77)
- (98, 90)
- (99, 56)
- (99, 111)
- (101, 29)
- (101, 138)
- (104, 47)
- (104,

In [9]:
def elliptic_add(p1, p2, a, b, p):
    # Handle the identity element cases (point at infinity)
    if p1 == "infinity":
        return p2
    if p2 == "infinity":
        return p1
    if p1 == p2:
        # Use the doubling formula
        if p1[1] == 0:  # The tangent is vertical, so the result is the point at infinity
            return "infinity"
        else:
            lam = (3 * p1[0]**2 + a) * sp.mod_inverse(2 * p1[1], p) % p
    else:
        # Use the addition formula
        if p1[0] == p2[0]:
            return "infinity"
        else:
            lam = (p2[1] - p1[1]) * sp.mod_inverse(p2[0] - p1[0], p) % p

    x3 = (lam**2 - p1[0] - p2[0]) % p
    y3 = (lam * (p1[0] - x3) - p1[1]) % p
    return (x3, y3)

def elliptic_mult(k, p1, a, b, p):
    r = "infinity"
    while k > 0:
        if k & 1:
            r = elliptic_add(r, p1, a, b, p)
        p1 = elliptic_add(p1, p1, a, b, p)
        k >>= 1
    return r

# P-256 curve parameters
a = -3
b = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B
p = 2**256 - 2**224 + 2**192 + 2**96 - 1

# Points provided by the user
P = (5139147556, 84992513513819837562679639329935059290744880011754896240095105442438501460580)
Q = (957, 106068017933963908069108378434410615354534760345589368572553274269533297279953)
R = (39147563, 86800512523116406738886579531519493100624547860931014107623369720327567099304)

# Perform the operations
P_plus_Q = elliptic_add(P, Q, a, b, p)
P17 = elliptic_mult(17, P, a, b, p)
P_plus_29Q = elliptic_add(P, elliptic_mult(29, Q, a, b, p), a, b, p)
P_plus_Q_plus_R = elliptic_add(P_plus_Q, R, a, b, p)
P5 = elliptic_mult(5, P, a, b, p)
P5_plus_47Q = elliptic_add(P5, elliptic_mult(47, Q, a, b, p), a, b, p)
P5_plus_47Q_plus_131R = elliptic_add(P5_plus_47Q, elliptic_mult(131, R, a, b, p), a, b, p)

In [10]:
print('          P + Q =', P_plus_Q)
print('            17P =', P17)
print('        P + 29Q =', P_plus_29Q)
print('      P + Q + R =', P_plus_Q_plus_R)
print('5P + 47Q + 131R =', P5_plus_47Q_plus_131R)

          P + Q = (70080563902474660685014643574954286003305136526954517743428749117055603385364, 102817281760810435925299871406466683251351115534973220072977695964583963694190)
            17P = (6767482609142533773727106561141486021988565046507560472655419434998143805142, 111849450923862212349083160889673641820507651447758924512379621060157659132486)
        P + 29Q = (85770230561767378375651721422453530671836821484546392654290517587029149330503, 109289525563572793074808101103724966387806510899333588476506211211325308580228)
      P + Q + R = (10078325695723211088410985734864258926108196424648752194418952354939491493398, 83162779257944430915141041076225903572090104654488309266745131800763293112040)
5P + 47Q + 131R = (60003140733697602815862847258574814281333745187497680713195857338569721153623, 113621516247140099510908544142253158409830215142911456940902986653313431422677)


In [11]:
# Define the EllipticCurve class that will allow us to perform operations over a given elliptic curve
class EllipticCurve:
    def __init__(self, a, b, p):
        # Initialize the coefficients and modulus of the curve: y^2 = x^3 + ax + b mod p
        self.a = a
        self.b = b
        self.p = p

    # Define the addition of two points on the curve
    def add_points(self, P, Q):
        # If either point is the identity, return the other point
        if P == "infinity":
            return Q
        if Q == "infinity":
            return P
        
        # If points are inverses of each other, return the identity
        if P[0] == Q[0] and (P[1] != Q[1] or P[1] == 0):
            return "infinity"
        
        # Calculate the slope (lambda)
        if P != Q:
            # Slope for two distinct points
            numerator = Q[1] - P[1]
            denominator = mod_inverse(Q[0] - P[0], self.p)
        else:
            # Slope for point doubling
            numerator = 3 * P[0]**2 + self.a
            denominator = mod_inverse(2 * P[1], self.p)
        
        # Perform the addition
        lam = (numerator * denominator) % self.p
        x3 = (lam**2 - P[0] - Q[0]) % self.p
        y3 = (lam * (P[0] - x3) - P[1]) % self.p
        
        return (x3, y3)

    # Define scalar multiplication of a point on the curve
    def mult_point(self, k, P):
        # Start with the identity (zero)
        Q = "infinity"
        
        # Double and add method for fast scalar multiplication
        while k:
            if k & 1:
                Q = self.add_points(Q, P)
            P = self.add_points(P, P)
            k >>= 1
        
        return Q

In [23]:
# Define ElGamal encryption over elliptic curves
class ElGamalEC:
    def __init__(self, q, G):
        self.q = q
        self.G = G
        self.curve = EllipticCurve(a, b, p)

    def keygen(self, alpha):
        if not isprime(self.q):
            raise ValueError("q must be a prime number.")
        self.sk = alpha
        self.Pk = self.curve.mult_point(alpha, self.G)
        return self.Pk, self.sk

    def encrypt(self, Pk, M):
        if not isinstance(M, tuple) or not len(M) == 2:
            raise TypeError("The message must be a point on the curve.")
        self.beta = randint(2, self.q - 1)
        self.V = self.curve.mult_point(self.beta, self.G)
        self.C = self.curve.add_points(M, self.curve.mult_point(self.beta, Pk))
        return self.V, self.C

    def decrypt(self, sk, V, C):
        sV = self.curve.mult_point(sk, V)
        # To find the inverse of sV, we negate its y-coordinate
        inv_sV = (sV[0], -sV[1] % self.curve.p)
        M_dec = self.curve.add_points(C, inv_sV)
        return M_dec

# Function to perform the operations required by the user
def perform_elgamal_ec_operations(alpha, M):

    a = -3
    b = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B
    p = 2**256 - 2**224 + 2**192 + 2**96 - 1  # The prime specifying the field size
    G = (0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296, 
         0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162cbf3e0f485a1e7169f34a31d76bfb3c)  # The base point (generator)
    q = p  # Assuming the order of the curve is p for simplicity; in reality, it would be a specific prime number representing the order of G

    # Create an instance of ElGamalEC with P-256 parameters
    egec = ElGamalEC(q, G)
    
    # Generate public and private keys
    Pk, sk = egec.keygen(alpha)
    
    # Encrypt the message point M
    V, C = egec.encrypt(Pk, M)
    
    # Decrypt the cipher back to the original message point
    M_dec = egec.decrypt(sk, V, C)
    
    # Assert to ensure the decryption is correct (for validation purposes)
    assert M == M_dec

    # Display the results
    print(f"(Pk) = {Pk}")
    print(f"(Sk) = {sk}")
    print(f"(beta, V, C) = {egec.beta, V, C}")
    print(f"M = {M_dec}")

    return (Pk, sk), (egec.beta, V, C), M_dec

In [24]:
# Example usage
alpha = 8043
M = (681515878241, 42867525214630099613645669273136473583026733865738235463867590073359574198973)

# Call the function with the given alpha and message point M
result = perform_elgamal_ec_operations(alpha, M)

(Pk) = (6260221265334454946073761813432262171617937705296575886744944238122303345810, 59880538412579264570600921042171759603764220585062289081256104934210992019803)
(Sk) = 8043
(beta, V, C) = (53412166207204606659017027724071016934317844131590157638410702467488055597115, (25643128626267729875227568681472805154752631262928854643397585881377414929238, 77882110148078180128918957248838408826969640438270997488002884594155651553117), (303823125751233496397332436515930378759204745428360352789166475438875305973, 58838979386543597102947464175620335114752870160505582124841002101788220063824))
M = (681515878241, 42867525214630099613645669273136473583026733865738235463867590073359574198973)
